In [1]:
!pip install selenium
!pip install bs4

     |████████████████████████████████| 9.9 MB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 400 kB 51.2 MB/s eta 0:00:01
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
  Using cached exceptiongroup-1.1.3-py3-none-any.whl (14 kB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached sniffio-1.3.0-py3-none-any.whl (10 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=b7ec8af611c90a3885f6a5e31a1459396a525227c135c751da1da025c6b7545d
  Stored in directory: /storage/home/pgajo/.cache/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
import time

# Initialize WebDriver
driver = webdriver.Chrome(executable_path="path/to/chromedriver")

# DataFrame to store ingredients and quantities
df = pd.DataFrame(columns=[f'i_{i}' for i in range(1, 51)] + [f'q_{i}' for i in range(1, 51)])

# Function to scrape a single page for its recipes
def scrape_page(page_url):
    global df
    driver.get(page_url)
    time.sleep(2)  # Give time for the page to load
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Look for the gz-list-ingredients section
    ingredient_list = soup.find('dl', {'class': 'gz-list-ingredients'})
    
    if ingredient_list:
        i_list = []
        q_list = []
        
        ingredients = ingredient_list.find_all('dd', {'class': 'gz-ingredient'})
        
        for ing in ingredients:
            ingredient = ing.find('a').text if ing.find('a') else ''
            quantity = ing.find('span').text if ing.find('span') else ''
            
            i_list.append(ingredient)
            q_list.append(quantity)
            
        # Padding lists to have uniform size
        i_list += [''] * (50 - len(i_list))
        q_list += [''] * (50 - len(q_list))
        
        # Appending to DataFrame
        df = df.append(pd.Series(i_list + q_list, index=df.columns), ignore_index=True)

# First URL to start scraping from
start_url = "https://ricette.giallozafferano.it/"

# TODO: Write code to iterate through all recipe pages on the website
# For demonstration, we are only scraping the first page
scrape_page(start_url)

# Save to CSV
df.to_csv('/home/pgajo/working/food/giallo-zafferano-corpus/ingredients.csv', index=False)
